# 0. Setting

- 라이브러리 설정

In [53]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import functional as F
from torch import nn

# 1. 데이터 준비

In [54]:
data = [
    'drink cold milk',
    'drink cold water',
    'drink cold cola',
    'drink sweet juice',
    'drink sweet cola',
    'eat delicious bacon',
    'eat sweet mango',
    'eat delicious cherry',
    'eat sweet apple',
    'juice with sugar',
    'cola with sugar',
    'mango is fruit',
    'apple is fruit',
    'cherry is fruit',
    'Berlin is Germany',
    'Boston is USA',
    'Mercedes from Germany',
    'Mercedes is car',
    'Ford from USA',
    'Ford is car'
]

# 2. 전처리

## 2-1. 단어 리스트 생성

In [55]:
word_sequence = [sequence.split() for sequence in data]
word_list = list(set(" ".join(data).split()))
word2index = {key : idx for idx, key in enumerate(word_list) if len(key) > 1}
vocab_size = len(word_list)

# 2-2. Dataset 생성

#### Window Size 정의

In [74]:
window_size = 2
batch_size = 10

### 2-2-1. Skip-gram Dataset 생성

### 페어 생성

- SkipGram_pair 함수 정의

In [57]:
def SkipGram_pair(window_size, word_sequence) :
    pair = []
    for sequence in word_sequence :
        for i in range(len(sequence)) :
            context = word2index[sequence[i]]
            target = []
            for j in range(window_size, -1, -1) :
                if i - j >= 0 : target.append(word2index[sequence[i-j]]) # 과거 문자 삽입
            for j in range(1, window_size + 1) :
                if i + j < len(sequence) : target.append(word2index[sequence[i+j]] ) # 미래 문자 삽입
            # 페어 생성
            for t in target : pair.append([context, t])

    x_train, y_train = list(zip(*pair))
    return torch.LongTensor(list(x_train)), torch.LongTensor(list(y_train))

In [58]:
x_train, y_train = SkipGram_pair(window_size, word_sequence)

- DataLoader 생성

In [59]:
train_dataset = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

# 3. Modeling

## 3-1. Skip-gram Modeling

In [60]:
class SkipGram(nn.Module) :

    def __init__(self, vocab_size, dimention_size) :
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, dimention_size)
        self.linear = nn.Linear(dimention_size, vocab_size, bias = False)
        self.activation = nn.LogSoftmax(dim = 1)

    def forward(self, X) :
        X = self.embeddings(X)
        X = self.linear(X)
        X = self.activation(X)
        return X

# 4. Training

## 4-1. Skip-Gram Training

- parameter setting

In [61]:
dimension_size = 10
epochs = 1000

- Training

In [73]:
model = SkipGram(vocab_size, dimension_size)
optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

for i in range(epochs+1) :
    for feature, label in train_dataloader :
        out = model(feature)
        loss = criterion(out, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 100 == 0 : print("epoch : {:d}, loss : {:0.3f}".format(i, loss))

epoch : 0, loss : 3.467
epoch : 100, loss : 2.753
epoch : 200, loss : 2.497
epoch : 300, loss : 2.499
epoch : 400, loss : 1.796
epoch : 500, loss : 1.778
epoch : 600, loss : 1.821
epoch : 700, loss : 1.662
epoch : 800, loss : 1.636
epoch : 900, loss : 2.296
epoch : 1000, loss : 1.238


# 5. Test

In [63]:
index2word = {value : key for key, value in word2index.items()}

## 5-1. 단어의 유사도 측정

### 5-1-1. 유사 단어 상위 3개 추출 함수 생성

In [64]:
def find_similarity(target_word) :
    target_word_embed = model.state_dict()['embeddings.weight'][word2index[target_word]]

    similarity = []
    for i in range(len(word2index)):
        if target_word != index2word[i]:
            similarity.append(( i, F.cosine_similarity(target_word_embed.unsqueeze(0), model.state_dict()['embeddings.weight'][i].unsqueeze(0)).item()))
        else:
            similarity.append((i, -1)) # target_word와 동일 단어는 -1 처리

    # 유사도 내림차순 정렬
    similarity.sort(key = lambda x : -x[1])

    # 인덱스를 단어로 변환
    print(f'{target_word}와 유사한 단어:')
    for i in range(3) :
        print(f'{i+1}위 : {index2word[similarity[i][0]]}({similarity[i][1]})')

### 5-2-2. 결과 확인

In [83]:
find_similarity("cola")

cola와 유사한 단어:
1위 : juice(0.5467252731323242)
2위 : apple(0.40455394983291626)
3위 : is(0.3187265992164612)
